In [43]:
# импорт необходимых библиотек
import pandas as pd

import requests

In [24]:
start_date = input('Введите дату начала в формате дд.мм.гггг')
finish_date = input('Введите дату конца в формате дд.мм.гггг')  # просим пользователя ввести даты начала и конца периода

Введите дату начала в формате дд.мм.гггг12.10.2022
Введите дату конца в формате дд.мм.гггг18.10.2022


In [25]:
start_date = pd.to_datetime(start_date, format='%d.%m.%Y')
finish_date = pd.to_datetime(finish_date, format='%d.%m.%Y')  # приводим введенные даты в формат даты

In [26]:
start_date = start_date.strftime('%m.%d.%Y')
finish_date = finish_date.strftime('%m.%d.%Y')  # приводим даты к формату, который требует сайт

In [27]:
# отправляем запрос на сайт
file = requests.get(f'https://cbbh.ba/CurrencyExchange/GetJsonForPeriod?dateFrom={start_date}&dateTo={finish_date}')

In [44]:
data = file.json()  #сохраняем полученные данные в json

In [33]:
df = pd.DataFrame(columns=('Дата', 'Курс'))  # делаем заготовку под будущий датафрейм

In [34]:
# нам из всех полученных данных нужна только дата и курс к доллару, поэтому сохраняем эти значения в списки
dates = []
currencies = []
for i in range(len(data)):
    dates.append(data[i]['Date'])
    currencies.append(data[i]['CurrencyExchangeItems'][13]['Middle'])

In [35]:
# передаем сохраненные значения в столбцы датафрейма
df['Дата'] = dates
df['Курс'] = currencies

In [36]:
# приводим все к правильному типу данных
df['Дата'] = pd.to_datetime(df['Дата'])
df['Курс'] = df['Курс'].str.replace(',', '.')
df['Курс'] = df['Курс'].astype(float)

In [40]:
# приводим дату к формату дд/мм/гггг
df['Дата'] = df['Дата'].dt.strftime('%d/%m/%Y')

In [41]:
df

,Дата,Курс
0,12/10/2022,2.011550
1,13/10/2022,2.015073
2,14/10/2022,2.008245
3,15/10/2022,2.012792
4,18/10/2022,2.008245


In [42]:
# сохраняем в excel
df.to_excel('cbbh.xlsx', index=False)